In [40]:
from ngsolve.webgui import Draw

import numpy as np
import ngsolve as ng
import dill
import pickle

motor_npz = np.load('../meshes/motor.npz', allow_pickle = True)

geoOCC = motor_npz['geoOCC'].tolist()
m = motor_npz['m']; m_new = m
j3 = motor_npz['j3']

geoOCCmesh = geoOCC.GenerateMesh()
mesh = ng.Mesh(geoOCCmesh)
# mesh.Refine()

# Draw(mesh)

In [41]:
import sys
sys.path.insert(0,'../../') # adds parent directory

import pde
order = 2

MESH = pde.mesh.netgen(mesh.ngmesh)

phi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'phi', order = x)
curlphi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'curlphi', order = x)

D = lambda x : pde.int.assemble(MESH, order = x)

myM = lambda x: phi_Hcurl(x)[0] @ D(x) @ phi_Hcurl(x)[0].T + \
                phi_Hcurl(x)[1] @ D(x) @ phi_Hcurl(x)[1].T

myK = lambda x: curlphi_Hcurl(x) @ D(x) @ curlphi_Hcurl(x).T

R = pde.tools.tree_cotree_gauge(MESH)

In [204]:
HCurl = ng.HCurl(mesh, order = 2, nograds = True, dirichlet = 'stator_outer')

u,v = HCurl.TnT()

K = ng.BilinearForm(HCurl)
K += (ng.curl(u)*ng.curl(v)+1e-6*u*v)*ng.dx
K.Assemble()

f = ng.LinearForm(HCurl)
f += ng.CF(1)*ng.curl(v)*ng.dx
# f += ng.CF((1,1))*v*ng.dx
f.Assemble()

# rows,cols,vals = K.mat.COO()
# import scipy.sparse as sp
# ngK = sp.csr_matrix((vals,(rows,cols)))

# np.linalg.matrix_rank(ngK.A)
# print(ngK.shape)

x = ng.GridFunction(HCurl)
x.vec.data += K.mat.Inverse(HCurl.FreeDofs())*f.vec

# Draw(x)

In [205]:
from mst import *
g = Graph(MESH.np)

edges = MESH.EdgesToVertices[:,:2]

for i in range(edges.shape[0]): g.addEdge(edges[i,0],edges[i,1],i)

g.KruskalMST()

indices = np.array(g.MST)[:,2]
indices

# [i for i in indices]

array([   0,    1,    2,    3,    4,    5,    6,    8,    9,   10, ..., 9880, 9906, 9911, 9915, 9921, 9928, 9932, 9948, 9953, 9960], dtype=int64)

In [206]:
import pyngcore as pyng
pyBA = pyng.BitArray(len(HCurl.FreeDofs()))
pyBA.Clear()

for _,j in enumerate(indices): pyBA.Set(j)
print(pyBA)

0: 11111110111111111111111111110011111111111111111111
50: 00111111111001111111000010100110111111111111111111
100: 11101010110110100111111111111111111111111111111111
150: 11111111111111111111111111111111110111111111111011
200: 11111111010111111111111111111111111111011110101101
250: 11111111111111111111111111101010110111111111111111
300: 11111111111101010110111111111111011111111111111101
350: 01100111111111111111111111111110101110011011111111
400: 11110111111000000110101101011101011011110101101110
450: 00100111010110101101011010110101101011010110101101
500: 01101011011011010110101101011010110101101011010110
550: 10110101101011010110101101011011011010110101101011
600: 01011010110101101011010110101101011010110110110101
650: 10101101011010110101101011010110101101011010110101
700: 10101101101101011010110101101011010110101101011010
750: 11010110101101011011011010110101101011010110101101
800: 01101011010110101101011010110101101011011011010110
850: 1011010110101101011010110101101011010110101101

23328